In [3]:
import boto3
from botocore.exceptions import NoCredentialsError

def download_file_from_s3(bucket, url, item, region):
    download_url = url.format(item, item)

    # Create a file to save the downloaded content
    try:
        with open(item, 'wb') as file:

            # Create a new session with S3 credentials
            session = boto3.session.Session(
                aws_access_key_id='GTFCGSSDTPICR71GOL6C',
                aws_secret_access_key='9gVCJIDVIMYHbAWDvYP8HJNxuaLoiWtQxQClSBUk',
                region_name=region
            )

            # Create a new S3 client
            s3_client = session.client('s3')

            # Perform the request to download the file
            try:
                response = s3_client.get_object(Bucket=bucket, Key=item)
                file.write(response['Body'].read())
                return file.name
            except NoCredentialsError:
                return "Error: No credentials provided for S3 access."
            except Exception as e:
                return f"Error downloading file: {str(e)}"
    except IOError as e:
        return f"Error creating file: {str(e)}"

# Usage



In [6]:
region = "us-east-1"
bucket = "scans"
item = "112849"
# S3Host: https://swift.wildberries.ru/scans/%s/%s.jpg
url = "https://swift.wildberries.ru/scans/{}/{}.jpg"

downloaded_file = download_file_from_s3(bucket, url, item, region)
print(downloaded_file)

Error downloading file: An error occurred (InvalidAccessKeyId) when calling the GetObject operation: The AWS Access Key Id you provided does not exist in our records.


In [10]:
import boto3
from botocore.config import Config

def create_s3_client(access_key, secret_key, region, bucket, use_ssl):
    # Configure the boto3 session
    session = boto3.session.Session(
        aws_access_key_id=access_key,
        aws_secret_access_key=secret_key,
        region_name=region
    )

    # Create an S3 client with the specified configurations
    s3_client = session.client(
        's3',
        config=Config(signature_version='s3v4', s3={'addressing_style': 'path' if use_ssl else 'virtual'}),
        use_ssl=use_ssl
    )

    return s3_client

# Example usage
appCfg = {
    "S3AccessKey": "GTFCGSSDTPICR71GOL6C",
    "S3SecretKey": "9gVCJIDVIMYHbAWDvYP8HJNxuaLoiWtQxQClSBUk",
    "S3Region": "us-east-1",
    "S3Bucket": "scans",
    "UseSSL": True  # or False, depending on your requirement
}

s3_client = create_s3_client(
    appCfg["S3AccessKey"],
    appCfg["S3SecretKey"],
    appCfg["S3Region"],
    appCfg["S3Bucket"],
    appCfg["UseSSL"]
)

In [11]:
s3_client.list_buckets()

ClientError: An error occurred (InvalidAccessKeyId) when calling the ListBuckets operation: The AWS Access Key Id you provided does not exist in our records.

In [12]:
import boto3
from botocore.exceptions import ClientError

def create_s3_session(access_key, secret_key, region, service_url, force_path):
    try:
        # Configure the boto3 session
        session = boto3.session.Session(
            aws_access_key_id=access_key,
            aws_secret_access_key=secret_key,
            region_name=region
        )

        # Create an S3 client with specific configuration
        s3_client = session.client(
            's3',
            endpoint_url=service_url,
            config=boto3.session.Config(s3={'addressing_style': 'path' if force_path else 'virtual'}),
            use_ssl=service_url.startswith('https://')
        )

        return s3_client
    except ClientError as e:
        raise Exception(f"Failed to create S3 session: {str(e)}")


In [15]:

# Example usage
access_key = 'GTFCGSSDTPICR71GOL6C'
secret_key = '9gVCJIDVIMYHbAWDvYP8HJNxuaLoiWtQxQClSBUk'
region = 'us-east-1'
service_url = 'https://swift.wildberries.ru'
force_path = True  # or False

s3_client = create_s3_session(access_key, secret_key, region, service_url, force_path)

In [ ]:
results = s3_client.list_buckets()

In [25]:
results = s3_client.list_objects_v2(Bucket='scans')

In [28]:
len(results['Contents'])

998

In [30]:
s3_client.get_object(Bucket='scans', Key='843632/photo.jpg')

{'ResponseMetadata': {'RequestId': 'tx00000000000000000fc5c-0065ae886d-1a4e402d-datapro',
  'HostId': '',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-length': '24682',
   'accept-ranges': 'bytes',
   'last-modified': 'Sun, 03 Sep 2023 01:54:01 GMT',
   'x-rgw-object-type': 'Normal',
   'etag': '"482a15844e2010e832eb87ee75441bfd"',
   'x-amz-request-id': 'tx00000000000000000fc5c-0065ae886d-1a4e402d-datapro',
   'content-type': 'binary/octet-stream',
   'date': 'Mon, 22 Jan 2024 15:23:25 GMT'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2023, 9, 3, 1, 54, 1, tzinfo=tzutc()),
 'ContentLength': 24682,
 'ETag': '"482a15844e2010e832eb87ee75441bfd"',
 'ContentType': 'binary/octet-stream',
 'Metadata': {},
 'Body': <botocore.response.StreamingBody at 0x1fe267f5360>}

In [ ]:
s3_client.list_objects_v2(Bucket='scans')